# To Do
- implement modulus for heading and wheel angles
- add disturbances

# Questions
- RK vs Euler for simulation
- how to implement mod for angles
- 

# System Dynamics
Parameters:

- $ r $: Radius of the wheels
- $ L $: Distance from wheel to wheel (half the body width)

State Variables:

- $ x $: $ x $-position of the rover's center of mass
- $ y $: $ y $-position of the rover's center of mass
- $ \psi $: Absolute heading angle of the rover (orientation of the body)
- $ \phi_L $: Absolute angle of rotation of the left wheel
- $ \phi_R $: Absolute angle of rotation of the right wheel

Inputs:

- $ \dot{\phi}_L $: Angular velocity of the left wheel
- $ \dot{\phi}_R $: Angular velocity of the right wheel

The system of differential equations describing the rover is:
$$
\begin{align}
\dot{x} &= \frac{r}{2} (\dot{\phi}_R + \dot{\phi}_L) \cos(\psi) \\
\dot{y} &= \frac{r}{2} (\dot{\phi}_R + \dot{\phi}_L) \sin(\psi) \\
\dot{\psi} &= \frac{r}{L} (\dot{\phi}_R - \dot{\phi}_L) \\
\dot{\phi}_L &= u_L \\
\dot{\phi}_R &= u_R
\end{align}
$$

We define our state and input vectors as follows:
$$
q = \begin{bmatrix}
            x \\
            y \\
            \psi \\
            \phi_L \\
            \phi_R
        \end{bmatrix}, 
u = \begin{bmatrix}
        u_L \\
        u_R
    \end{bmatrix} 
    = \begin{bmatrix}
        \dot{\phi}_L \\
        \dot{\phi}_R
    \end{bmatrix} 
$$

The continuous time state space dynamics are:
$$ 
\begin{align}
\dot{q} &= f(q, u) \\
        &= \begin{bmatrix}
                \frac{r}{2} \cos(\psi) & \frac{r}{2} \cos(\psi) \\
                \frac{r}{2} \sin(\psi) & \frac{r}{2} \sin(\psi) \\
                -\frac{r}{L} & \frac{r}{L} \\
                1 & 0 \\
                0 & 1
            \end{bmatrix} 
            \begin{bmatrix}
                u_L \\
                u_R
            \end{bmatrix} 
\end{align}
$$

We can discretize the system:
$$
\begin{align}
    q_{k+1} &= q_k + T_s f(q_k, u_k) \\
            &= q_k + T_s \begin{bmatrix}
                            \frac{r}{2} \cos(\psi) & \frac{r}{2} \cos(\psi) \\
                            \frac{r}{2} \sin(\psi) & \frac{r}{2} \sin(\psi) \\
                            -\frac{r}{L} & \frac{r}{L} \\
                            1 & 0 \\
                            0 & 1
                        \end{bmatrix} 
                        \begin{bmatrix}
                            u_L \\
                            u_R
                        \end{bmatrix} 
\end{align}
$$


# Functions

#### CFTOC Function definition

In [1]:
# ! pip install numpy
# ! pip install matplotlib
# ! pip install scipy
from __future__ import division
import pyomo.environ as pyo
import numpy as np
# import polytope as pt
import matplotlib.pyplot as plt


available_solvers = pyo.SolverFactory._cls
print(list(available_solvers.keys()))


['_neos', 'cbc', '_cbc_shell', '_mock_cbc', 'glpk', '_glpk_shell', '_mock_glpk', 'cplex', '_cplex_shell', '_mock_cplex', 'gurobi_direct', 'asl', '_mock_asl', 'gurobi', '_gurobi_nl', '_gurobi_shell', '_gurobi_file', 'baron', 'py', 'scip', 'conopt', 'xpress', 'ipopt', 'gurobi_persistent', 'cplex_direct', 'cplex_persistent', 'gams', '_gams_direct', '_gams_shell', 'mosek', 'mosek_direct', 'mosek_persistent', 'xpress_direct', 'xpress_persistent', 'sas', '_sas94', '_sascas', 'cp_optimizer', 'mpec_nlp', 'mpec_minlp', 'path', 'appsi_gurobi', 'appsi_cplex', 'appsi_ipopt', 'appsi_cbc', 'appsi_highs', 'appsi_maingo', 'gdpopt', 'gdpopt.gloa', 'gdpopt.lbb', 'gdpopt.loa', 'gdpopt.ric', 'gdpopt.enumerate', 'contrib.gjh', 'mindtpy', 'mindtpy.oa', 'mindtpy.ecp', 'mindtpy.goa', 'mindtpy.fp', 'multistart', 'cyipopt', 'scipy.fsolve', 'scipy.root', 'scipy.newton', 'scipy.secant-newton', 'ipopt_v2', 'gurobi_v2', 'gurobi_direct_v2', 'trustregion']


In [4]:
# GLOBAL VARIABLES
r_cm = 3   # cm 
L_cm = 5  # cm
Ts = 0.2   # s
MAX_deltaU = 1
import logging
from pyomo.util.infeasible import log_infeasible_constraints

# Set the logger to DEBUG level
logging.basicConfig(level=logging.DEBUG)


def solve_cftoc_batch(q0, qf, N, r_cm, L_cm, Ts):
  r = r_cm
  L = L_cm

  nq = 5         # number of states (x,y,theta)
  nu = 2         # number of inputs u1=v, u2=omega

  model = pyo.ConcreteModel()
  model.tidx = pyo.Set(initialize=range(0, N+1))    # length of finite optimization problem for states
  model.tuidx = pyo.Set(initialize=range(0, N))     # length of finite optimization problem for input
  model.qidx = pyo.Set(initialize=range(0, nq))
  model.uidx = pyo.Set(initialize=range(0, nu))

  # Create state and input variables trajectory:
  model.q = pyo.Var(model.qidx, model.tidx)
  model.u = pyo.Var(model.uidx, model.tuidx)

  # Cost function: penalize inputs
  def cost_rule(model):
    u_cost = 0
    q_cost = 0

    for i in model.uidx:
      for t in model.tuidx:
        u_cost += model.u[i, t]**2

    for t in model.tidx:
      q_cost += (model.q[0, t] - qf[0])**2 + (model.q[1, t] - qf[1])**2
    
    return u_cost + q_cost

  model.cost = pyo.Objective(rule=cost_rule, sense=pyo.minimize)

  # Initial conditions, fix the value:
  for i in model.qidx:
    model.q[i, 0].fix(q0[i])

  # Constraints:
  model.sys_dyn0 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[0, t+1] == model.q[0, t] + Ts* (r/2)*(model.u[0,t] + model.u[1,t])*pyo.cos(model.q[2,t])
                                    if t < N else pyo.Constraint.Skip)
  model.sys_dyn1 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[1, t+1] == model.q[1, t] + Ts* (r/2)*(model.u[0,t] + model.u[1,t])*pyo.sin(model.q[2,t])
                                    if t < N else pyo.Constraint.Skip)
  model.sys_dyn2 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[2, t+1] == model.q[2, t] + Ts* (r/L)*(-model.u[0,t] + model.u[1,t])
                                    if t < N else pyo.Constraint.Skip)
  model.sys_dyn3 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[3, t+1] == model.q[3, t] + Ts* model.u[0,t]
                                    if t < N else pyo.Constraint.Skip)
  model.sys_dyn4 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[4, t+1] == model.q[4, t] + Ts* model.u[1,t]
                                    if t < N else pyo.Constraint.Skip)
  
  # Box input constraints:
  max_omega = 1
  model.input_constraint01 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.u[0, t] <= max_omega
                                    if t <= N-1 else pyo.Constraint.Skip)
  model.input_constraint02 = pyo.Constraint(model.tuidx, rule=lambda model, t: -model.u[0, t] <= max_omega
                                    if t <= N-1 else pyo.Constraint.Skip)
  model.input_constraint11 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.u[1, t] <= max_omega
                                    if t <= N-1 else pyo.Constraint.Skip)
  model.input_constraint12 = pyo.Constraint(model.tuidx, rule=lambda model, t: -model.u[1, t] <= max_omega
                                    if t <= N-1 else pyo.Constraint.Skip)
  model.input_constraint13 = pyo.Constraint(model.tuidx, rule=lambda model, t: (model.u[0,t]-model.u[0,t-1])**2 <= MAX_deltaU if t > 0 and t < N-1 else pyo.Constraint.Skip)
  model.input_constraint14 = pyo.Constraint(model.tuidx, rule=lambda model, t: (model.u[1,t]-model.u[1,t-1])**2 <= MAX_deltaU if t > 0 and t < N-1 else pyo.Constraint.Skip)
  
  # will only constrain y, psi, and phiL phiR (x is free)
  model.final_constraint1 = pyo.Constraint(expr=model.q[1, N] == qf[1])
  
  # model.k2 = pyo.Var(domain=pyo.Integers)
  # model.k3 = pyo.Var(domain=pyo.Integers)
  # model.k4 = pyo.Var(domain=pyo.Integers)
  # model.final_constraint2 = pyo.Constraint(expr=model.q[2, N] == qf[2] + 2*np.pi*model.k2)
  # model.final_constraint3 = pyo.Constraint(expr=model.q[3, N] == qf[3] + 2*np.pi*model.k3)
  # model.final_constraint4 = pyo.Constraint(expr=model.q[4, N] == qf[4] + 2*np.pi*model.k4)

  model.final_constraint2 = pyo.Constraint(expr=model.q[2, N] == qf[2])
  model.final_constraint3 = pyo.Constraint(expr=model.q[3, N] == qf[3])
  model.final_constraint4 = pyo.Constraint(expr=model.q[4, N] == qf[4])

  # model.final_constraint2 = pyo.Constraint(expr=pyo.cos(model.q[2, N]) == pyo.cos(qf[2]))
  # # model.final_constraint21 = pyo.Constraint(expr=pyo.sin(model.q[2, N]) == pyo.sin(qf[2]))
  # model.final_constraint3 = pyo.Constraint(expr=pyo.cos(model.q[3, N]) == pyo.cos(qf[3]))
  # # model.final_constraint31 = pyo.Constraint(expr=pyo.sin(model.q[3, N]) == pyo.sin(qf[3]))
  # model.final_constraint4 = pyo.Constraint(expr=pyo.cos(model.q[4, N]) == pyo.cos(qf[4]))
  # model.final_constraint41 = pyo.Constraint(expr=pyo.sin(model.q[4, N]) == pyo.sin(qf[4]))

  # Now we can solve:
  solver = pyo.SolverFactory('ipopt')
  results = solver.solve(model)
  
  feas = str(results.solver.termination_condition) == "optimal"
  qOpt = np.asarray([model.q[:,t]() for t in model.tidx]).T
  uOpt = np.asarray([model.u[:,t]() for t in model.tuidx]).T
  JOpt = model.cost()
  log_infeasible_constraints(model)


  return [feas, qOpt, uOpt, JOpt, model, results]


# Simulations/Animations

### Batch optimization for trajectory generation

In [5]:
q0 = [0,0, 0, np.pi, 0]
qf=[0,0,0,0,0]

N_batch = 10
[feas, q_traj_batch, u_traj_batch, JOpt, model, results] = solve_cftoc_batch(q0, qf, N_batch, r_cm, L_cm, Ts)

#print out the final state
print(q_traj_batch[:,-1])
print(JOpt)
# print(feas)
# print(q_traj_batch)
model.pprint()



DEBUG:pyomo.core:Constructing ConcreteModel 'ConcreteModel', from data=None
DEBUG:pyomo.core:Constructing AbstractOrderedScalarSet 'tidx' on [Model] from data=None
DEBUG:pyomo.core:Constructing Set, name=tidx, from data=None
DEBUG:pyomo.core:Constructed component ''[Model].tidx'':
tidx : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

DEBUG:pyomo.core:Constructing AbstractOrderedScalarSet 'tuidx' on [Model] from data=None
DEBUG:pyomo.core:Constructing Set, name=tuidx, from data=None
DEBUG:pyomo.core:Constructed component ''[Model].tuidx'':
tuidx : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

DEBUG:pyomo.core:Constructing AbstractOrderedScalarSet 'qidx' on [Model] from data=None
DEBUG:pyomo.core:Constructing Set, name=qidx, from data=None
DEBUG:pyomo.core:Constructed component ''[Mod

[-2.99760872e+00  5.98901153e-22  1.03997322e-01  1.02567599e-01
 -1.08763066e-11]
46.93917728189596
4 Set Declarations
    qidx : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    tidx : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    tuidx : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    uidx : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}

2 Var Declarations
    q : Size=55, Index=qidx*tidx
        Key     : Lower : Value                   : Upper : Fixed : Stale : Domain
         (0, 0) :  None :                       0 :  None :  True :  True :  Reals
         (0, 1) :  None :     -

In [ ]:
plt.figure()
plt.plot(q_traj_batch[0,:], q_traj_batch[1,:], 'ro-')
# Show the initial x,y position
plt.plot(q_traj_batch[0,0], q_traj_batch[1,0], 'go')
plt.xlim([min(q_traj_batch[0,:])-(r_cm+L_cm/2), max(q_traj_batch[0,:])+(r_cm+L_cm/2)])
plt.ylim([min(q_traj_batch[1,:])-(r_cm+L_cm/2), max(q_traj_batch[1,:])+(r_cm+L_cm/2)])

plt.xlabel('x')
plt.ylabel('y')
plt.title('qOpt')
plt.legend(['qOpt','qOpt(0)'])

# Plot inputs in two subplots
plt.figure()
plt.subplot(2,1,1)
plt.plot(range(N_batch), u_traj_batch[0,:], 'ro-')
plt.xlabel('Time')
plt.ylabel('u1')

plt.subplot(2,1,2)
plt.plot(range(N_batch), u_traj_batch[1,:], 'bo-')
plt.xlabel('Time')
plt.ylabel('u2')
plt.tight_layout()

# Plot all states
plt.figure()

# Subplot for x and y
plt.subplot(2, 1, 1)
plt.plot(range(N_batch+1), q_traj_batch[0,:], 'ro-', label='x')
plt.plot(range(N_batch+1), q_traj_batch[1,:], 'bo-', label='y')
plt.xlabel('Time')
plt.ylabel('x, y')
plt.legend()
plt.title('Position States')

# Subplot for psi, phiL, and phiR
plt.subplot(2, 1, 2)
plt.plot(range(N_batch+1), q_traj_batch[2,:], 'go-', label='psi')
plt.plot(range(N_batch+1), q_traj_batch[3,:], 'mo-', label='phiL')
plt.plot(range(N_batch+1), q_traj_batch[4,:], 'yo-', label='phiR')
plt.xlabel('Time')
plt.ylabel('psi, phiL, phiR')
plt.legend()
plt.title('Angle States')

plt.tight_layout()


### Animation

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
import numpy as np

def plotRoverState(ax, x, y, psi, phiL, phiR, L, r_ratio):
    r = r_ratio * L / 2
    dx = (L / 2) * np.cos(psi + np.pi / 2)
    dy = (L / 2) * np.sin(psi + np.pi / 2)
    
    #Show the full trajectory
    ax.plot(q_traj_batch[0,:], q_traj_batch[1,:], 'k-', alpha=0.5)

    #Plot the rover
    ax.plot([x - dx, x + dx], [y - dy, y + dy], 'k-', linewidth=3)
    ax.plot(x, y, 'ko')
    ax.quiver(x, y, np.cos(psi), np.sin(psi), scale=20)
    
    wheelPosL = (x + dx, y + dy)
    wheelPosR = (x - dx, y - dy)

    circle1 = Circle(wheelPosL, r, edgecolor='b', facecolor='none')
    circle2 = Circle(wheelPosR, r, edgecolor='r', facecolor='none')
    ax.add_patch(circle1)
    ax.add_patch(circle2)

    ax.plot([wheelPosL[0], wheelPosL[0] + r * np.cos(phiL)], 
            [wheelPosL[1], wheelPosL[1] + r * np.sin(phiL)], 'b-')
    ax.plot([wheelPosR[0], wheelPosR[0] + r * np.cos(phiR)], 
            [wheelPosR[1], wheelPosR[1] + r * np.sin(phiR)], 'r-')

def calculate_wheel_positions(trajectory, L):
    N = len(trajectory)
    print(np.shape(trajectory))
    wheelL_pos = np.zeros((N, 2))
    wheelR_pos = np.zeros((N, 2))

    for i in range(N):
        x, y, psi, _, _ = trajectory[i]
        dx = (L / 2) * np.cos(psi + np.pi / 2)
        dy = (L / 2) * np.sin(psi + np.pi / 2)

        wheelL_pos[i] = [x + dx, y + dy]
        wheelR_pos[i] = [x - dx, y - dy]

    return wheelL_pos, wheelR_pos

def animate_trajectory(q_traj, u_traj, qf, L):
    n_k = np.shape(u_traj)[0]
    print(n_k)
    eps = 0.001
    for i in range(n_k):
        if ((q_traj[i,0] - qf[0])**2<= eps) & ((q_traj[i,1]-qf[1])**2<=eps) & ((q_traj[i,2]-qf[2])**2<=eps) & ((q_traj[i,3]-qf[3])**2<=eps) & ((q_traj[i,4]-qf[4])**2<=eps) & (u_traj[i,0]**2 <= eps) & (u_traj[i,1]**2<= eps):
            n_k = i
            break
    print(n_k)        
    wheelL_pos, wheelR_pos = calculate_wheel_positions(q_traj, L)

    fig = plt.figure(figsize=(14, 6))
    ax1 = plt.subplot2grid((2, 4), (0, 0), colspan=2, rowspan=2)
    ax2 = plt.subplot2grid((2, 4), (0, 2))
    ax3 = plt.subplot2grid((2, 4), (0, 3))
    ax4 = plt.subplot2grid((2, 4), (1, 2))
    ax5 = plt.subplot2grid((2, 4), (1, 3))
    plt.tight_layout()

    def update(frame):
        ax1.clear()
        ax2.clear()
        ax3.clear()
        ax4.clear()
        ax5.clear()

        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax1.set_xlim([min(q_traj[:,0])-(r_cm+L_cm/2), max(q_traj[:,0])+(r_cm+L_cm/2)])
        ax1.set_ylim([min(q_traj[:,1])-(r_cm+L_cm/2), max(q_traj[:,1])+(r_cm+L_cm/2)])
        ax1.grid(True)
        ax1.set_aspect('equal')

        # Plot rover state
        plotRoverState(ax1, *q_traj[frame], L, 0.3)

        # Plot trajectory
        x, y, psi, phiL, phiR = q_traj.T
        ax2.plot(range(n_k), x[:n_k], '.-', label='X-pos')
        ax2.plot(range(n_k), y[:n_k], '.-', label='Y-pos')
        ax2.plot(range(n_k), psi[:n_k], '.-', label='Heading')
        ax2.axvline(frame, 0, 1)
        ax2.legend()

        # Plot wheel angles
        ax3.plot(range(n_k), phiL[:n_k], '.-b', label="Lwhl θ")
        ax3.plot(range(n_k), phiR[:n_k], '.-r', label="Rwhl θ")
        ax3.axvline(frame, 0, 1)
        ax3.legend()

        # Plot wheel positions
        ax4.plot(range(n_k), wheelL_pos[:n_k, 0], '.-b', label="Lwhl x")
        ax4.plot(range(n_k), wheelL_pos[:n_k, 1], '.-b', label="Lwhl y")
        ax4.plot(range(n_k), wheelR_pos[:n_k, 0], '.-r', label="Rwhl x")
        ax4.plot(range(n_k), wheelR_pos[:n_k, 1], '.-r', label="Rwhl y")
        ax4.axvline(frame, 0, 1)
        ax4.legend()

        # Plot inputs
        ax5.plot(range(n_k), u_traj[:n_k, 0], '.-b', label="uL")
        ax5.plot(range(n_k), u_traj[:n_k, 1], '.-r', label="uR")
        ax5.axvline(frame, 0, 1)
        ax5.legend()

        return ax1, ax2, ax3, ax4

    ani = FuncAnimation(fig, update, frames=n_k, repeat=False)
    return ani


ani = animate_trajectory(q_traj_batch.T, u_traj_batch.T, qf, L_cm)
# Display animation in Jupyter
from IPython.display import HTML
HTML(ani.to_jshtml())
 

### MPC
Simulate the MPC 1 step at a time

In [6]:

def solve_cftoc_mpc(q0, q_ref, u_ref, N, P, Q, R, r, L, Ts):
    r = r_cm
    L = L_cm

    nq = 5         # number of states (x,y,psi,phiL,phiR)
    nu = 2         # number of inputs u1=phiL_dot, u2=phiR_dot
    model = pyo.ConcreteModel()
    
    model.P = P
    model.Q = Q
    model.R = R
    model.N = N

    model.tidx = pyo.Set(initialize=range(0, N+1), ordered=True)  # length of finite optimization problem
    model.tuidx = pyo.Set(initialize=range(0, N), ordered=True)   # length of finite optimization problem
    model.qidx = pyo.Set(initialize=range(0, nq), ordered=True)
    model.uidx = pyo.Set(initialize=range(0, nu), ordered=True)

    # Create state and input variables trajectory:
    model.q = pyo.Var(model.qidx, model.tidx)
    model.u = pyo.Var(model.uidx, model.tuidx)

    # Constraints:
    model.init_condition = pyo.Constraint(model.qidx, rule=lambda model, i: model.q[i, 0] == q0[i])
    model.sys_dyn1 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[0, t+1] == model.q[0, t] + Ts* (r/2)*(model.q[3,t] + model.q[4,t])*pyo.cos(model.q[2,t])
                                    if t < N else pyo.Constraint.Skip)
    model.sys_dyn2 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[1, t+1] == model.q[1, t] + Ts* (r/2)*(model.q[3,t] + model.q[4,t])*pyo.sin(model.q[2,t])
                                    if t < N else pyo.Constraint.Skip)
    model.sys_dyn3 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[2, t+1] == model.q[2, t] + Ts* (r/L)*(model.q[4,t] - model.q[3,t])
                                    if t < N else pyo.Constraint.Skip)
    model.sys_dyn4 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[3, t+1] == model.q[3, t] + Ts* model.u[0,t]
                                    if t < N else pyo.Constraint.Skip)
    model.sys_dyn5 = pyo.Constraint(model.tidx, rule=lambda model, t: model.q[4, t+1] == model.q[4, t] + Ts* model.u[1,t]
                                    if t < N else pyo.Constraint.Skip)

    # Box input constraints:
    model.input_constraint01 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.u[0, t] <= 1
                                    if t <= N-1 else pyo.Constraint.Skip)
    model.input_constraint02 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.u[0, t] >= -1
                                    if t <= N-1 else pyo.Constraint.Skip)
    model.input_constraint11 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.u[1, t] <= 1
                                    if t <= N-1 else pyo.Constraint.Skip)
    model.input_constraint12 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.u[1, t] >= -1
                                    if t <= N-1 else pyo.Constraint.Skip)
    
    model.input_constraint13 = pyo.Constraint(model.tuidx, rule=lambda model, t: (model.u[0,t]-model.u[0,t-1])**2 <= MAX_deltaU if t > 0 else pyo.Constraint.Skip)
    model.input_constraint14 = pyo.Constraint(model.tuidx, rule=lambda model, t: (model.u[1,t]-model.u[1,t-1])**2 <= MAX_deltaU if t > 0 else pyo.Constraint.Skip)
    
    # Final state constraints
    # model.final_constraint = pyo.Constraint(model.qidx, rule=lambda model, i: model.q[i, N] == q_ref[i, N])

    # Cost Function
    def cost_rule(model):
        stage_state_cost = 0
        stage_input_cost = 0
        terminal_state_cost = 0
        for t in model.tidx:
            for i in model.qidx:
                if i < 2:
                    stage_state_cost += model.Q[i] * (model.q[i,t] - q_ref[i,t]) **2
                if i > 1:
                    stage_state_cost += model.Q[i] * ((pyo.cos(model.q[i, t]) - pyo.cos(q_ref[i,t]))**2 + (pyo.sin(model.q[i, t]) - pyo.sin(q_ref[i,t]))**2)
        
        for t in model.tuidx:
            for i in model.uidx:
                stage_input_cost += model.R[i] * (model.u[i,t] - u_ref[i,t]) **2

        for i in model.qidx:
            terminal_state_cost += model.P[i] * (model.q[i,model.N] - q_ref[i,model.N]) **2

        return stage_state_cost + stage_input_cost
    model.cost = pyo.Objective(rule=cost_rule, sense=pyo.minimize)

    # Now we can solve:
    solver = pyo.SolverFactory('ipopt')
    results = solver.solve(model)

    feas = str(results.solver.termination_condition) == "optimal"
    qOpt = np.asarray([[model.q[i,t]() for i in model.qidx] for t in model.tidx]).T
    uOpt = np.asarray([model.u[:,t]() for t in model.tuidx]).T
    JOpt = model.cost()
    # model.pprint()

    return [feas, qOpt, uOpt, JOpt, model, results]




In [ ]:
from scipy.integrate import solve_ivp

# Function to discretize the continuous time dynamics, using Runge-Kutta integration
def sysDyn_sim_RK_kplus1(q_k, u_k, r, L, Ts):
    def continuous_dynamics(t, q_k, u_k, r, L):
        dx = (r/2)*(u_k[0] + u_k[1])*np.cos(q_k[2])
        dy = (r/2)*(u_k[0] + u_k[1])*np.sin(q_k[2])
        dpsi = (r/L)*(-u_k[0] + u_k[1])
        dphiL = u_k[0]
        dphiR = u_k[1]

        dqdt = np.array([dx, dy, dpsi, dphiL, dphiR])
        return dqdt

    t_span = [0, Ts]
    
    # Using solve_ivp to integrate using 'RK45' (Runge-Kutta-Fehlberg)
    sol = solve_ivp(continuous_dynamics, t_span, q_k, args=(u_k, r, L), method='RK45')

    q_kplus1 = sol.y[:, -1]  # final state at time Ts
    return q_kplus1

# Function to discretize the continuous time dynamics, using Euler discretization
def sysDyn_sim_euler_kplus1(q_k, u_k, r, L, Ts):
    B = np.array([  [r/2*np.cos(q_k[2]), r/2*np.cos(q_k[2])], 
                    [r/2*np.sin(q_k[2]), r/2*np.sin(q_k[2])],
                    [-r/L, r/L], 
                    [1, 0], 
                    [0, 1]])
    q_kplus1 = q_k + Ts * B @ u_k

    return q_kplus1


#Compare the two methods by simulating the system
N_batch = 100
q0 = [0,0, 0, np.pi, 0]
q_sim_RK = np.zeros((5, N_batch+1))
q_sim_euler = np.zeros((5, N_batch+1))
q_sim_RK[:, 0] = q0
q_sim_euler[:, 0] = q0

#Use test inputs
u_test = np.vstack((np.ones(N_batch), np.zeros(N_batch)))
# [feas, q_traj_batch, u_traj_batch, JOpt, model, results] = solve_cftoc_batch(q0, qf, N_batch, r_cm, L_cm, Ts)
# u_test = u_traj_batch

# Roll the system forward
for k in range(N_batch):
    q_sim_RK[:, k+1] = sysDyn_sim_RK_kplus1(q_sim_RK[:, k], u_test[:, k], r_cm, L_cm, Ts)
    q_sim_euler[:, k+1] = sysDyn_sim_euler_kplus1(q_sim_euler[:, k], u_test[:, k], r_cm, L_cm, Ts)

# Plot the results
plt.figure(figsize=(8, 12))

plt.subplot(3,1,1)
plt.plot(q_sim_RK[0,:], q_sim_RK[1,:], 'ro-', label='RK45')
plt.plot(q_sim_euler[0,:], q_sim_euler[1,:], 'bo-', label='Euler')
plt.plot(q_traj_batch[0,:], q_traj_batch[1,:], 'k-', label='Batch')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Simulated trajectory')
plt.legend()

plt.subplot(3,1,2)
plt.plot(range(N_batch+1), q_sim_RK[2,:], 'ro-', label='RK45')
plt.plot(range(N_batch+1), q_sim_euler[2,:], 'bo-', label='Euler')
plt.xlabel('Time')
plt.ylabel('psi')
plt.legend()

plt.subplot(3,1,3)
plt.plot(range(N_batch), u_test[0,:], 'ro-', label='u1')
plt.plot(range(N_batch), u_test[1,:], 'bo-', label='u2')
plt.xlabel('Time')
plt.ylabel('Inputs')
plt.legend()

plt.tight_layout()





#### Simulate the full control system
1) Batch optimization to find trajectory to follow
2) MPC to follow path.  Simulate 1 step at a time and record.

In [10]:
N_batch = 100
N_horizon = 10
q0 = [0,0,0, np.pi, 2*np.pi]
qf = np.array([0, 0, 0, 0, 0])

# 0) Modify inputs
q0[2] = q0[2] % 2*np.pi
q0[3] = q0[3] % 2*np.pi
q0[4] = q0[4] % 2*np.pi

# 1) Batch trajectory calculation
[feas, q_traj_batch, u_traj_batch, JOpt, model, results] = solve_cftoc_batch(q0, qf, N_batch, r_cm, L_cm, Ts)

# 2) Simulate the system using the optimal inputs
nq = 5
nu = 2
P = np.array([1e3, 1e3, 1e3, 1e3, 1e3])
Q = np.array([1e3, 1e3, 1, 1, 1])
R = np.ones(nu)

q_MPC = np.zeros((nq, N_batch+1))
q_MPC[:, 0] = q0
u_MPC = np.zeros((nu, N_batch))

q_MPC_openloop = np.zeros((nq, N_batch+1, N_horizon+1))
u_MPC_openloop = np.zeros((nu, N_batch+1, N_horizon))

#Pad the reference trajectories.  For q, use the last state.  For u, use 0's
q_traj_batch = np.hstack((q_traj_batch, np.tile(q_traj_batch[:,-1], (N_horizon, 1)).T))
u_traj_batch = np.hstack((u_traj_batch, np.zeros((nu, N_horizon))))

# FOR REFERENCE: [feas, qOpt, uOpt, JOpt, model, results] = solve_cftoc_mpc(q0, q_ref, u_ref, N, Q, R, r, L, Ts)
for t_mpc in range(N_batch):
    # Define the reference trajectory for MPC horizon
    q_ref = q_traj_batch[:, t_mpc:t_mpc+N_horizon+1]
    u_ref = u_traj_batch[:, t_mpc:t_mpc+N_horizon]

    # Get the MPC open loop trajectory/input
    [feas, qOpt, uOpt, JOpt, model, results] = solve_cftoc_mpc(q_MPC[:, t_mpc], q_ref, u_ref, N_horizon, P, Q, R, r_cm, L_cm, Ts)
    q_MPC_openloop[:, t_mpc, :] = qOpt
    u_MPC_openloop[:, t_mpc, :] = uOpt

    # Apply the first input to the system, simulate forward
    u_MPC[:, t_mpc] = uOpt[:, 0]
    # q_MPC[:, t_mpc+1] = sysDyn_sim_euler_kplus1(q_MPC[:, t_mpc], u_MPC[:, t_mpc], r_cm, L_cm, Ts)
    q_MPC[:, t_mpc+1] = sysDyn_sim_RK_kplus1(q_MPC[:, t_mpc], u_MPC[:, t_mpc], r_cm, L_cm, Ts)

# print(f"Final state: {q_MPC[:,-1]}")
# # print(f"MPC Trajectory: {q_MPC}")
# print(f"MPC Inputs 0: {u_MPC[0,:]}")
# print(f"Batch inputs 0: {u_traj_batch[0,:]}")

# print(f"MPC Inputs 1: {u_MPC[1,:]}")
# print(f"Batch inputs 1: {u_traj_batch[1,:]}")


#### Plotting

In [ ]:
# Print the final state
print(f"Final state: {q_MPC[:, -1]}")
print(f"Final state (batch): {q_traj_batch[:, -1]}")


# Plot the closed-loop trajectory and the open-loop trajectories
# Also show the states in subplots
fig = plt.figure(figsize=(14, 6))  # Adjust the size as needed

# Large square axes
ax1 = plt.subplot2grid((2,4), (0, 0), colspan=2, rowspan=2)  # Spans all three rows in the first column

# Four smaller plots in a 2x2 grid on the side
ax2 = plt.subplot2grid((2, 4), (0, 2))  # First row, second column
ax3 = plt.subplot2grid((2, 4), (0, 3))  # First row, third column
ax4 = plt.subplot2grid((2, 4), (1, 2))  # Second row, second column
ax5 = plt.subplot2grid((2, 4), (1, 3))  # Second row, third column

#Plot the open-loop trajectories
for t in range(N_batch):
    open_loop = ax1.plot(q_MPC_openloop[0, t, :], q_MPC_openloop[1, t, :], 'b:.', alpha=0.1)

closed_loop = ax1.plot(q_MPC[0,:], q_MPC[1,:], 'ko-', label='Closed-loop')
batch_traj = ax1.plot(q_traj_batch[0,:], q_traj_batch[1,:], 'r-', label='Batch')

ax1.plot(q0[0], q0[1], 'ro', markersize=10) # Show the initial position
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.legend([open_loop[0], closed_loop[0], batch_traj],['Open-loop','Closed-loop','Batch Planned'])

#Position and heading
ax2.plot(range(N_batch+1), q_MPC[0,:], 'ro-', label='x')
ax2.plot(range(N_batch+1), q_MPC[1,:], 'bo-', label='y')
ax2.plot(range(N_batch+1), q_MPC[2,:], 'go-', label='psi')
ax2.set_xlabel('Time')
ax2.set_ylabel('x,y,psi')
ax2.legend()

#Wheel angles
ax3.plot(range(N_batch+1), q_MPC[3,:], 'ro-', label='phiL')
ax3.plot(range(N_batch+1), q_MPC[4,:], 'bo-', label='phiR')
ax3.set_xlabel('Time')
ax3.set_ylabel('phiL, phiR')
ax3.legend()

#x,y positions over time with open-loop trajectories
OL_length = q_MPC_openloop.shape[2]
for t in range(N_batch):
    ax4.plot(np.arange(OL_length)+t, q_MPC_openloop[0, t, :], 'r:.', alpha=0.1)
    ax4.plot(np.arange(OL_length)+t, q_MPC_openloop[1, t, :], 'b:.', alpha=0.1)
ax4.plot(range(N_batch+1), q_MPC[0,:], 'ro-', label='CL x')
ax4.plot(range(N_batch+1), q_MPC[1,:], 'bo-', label='CL y')
ax4.set_xlabel('Time')
ax4.set_ylabel('x,y')
ax4.legend()

#Inputs
ax5.plot(range(N_batch), u_MPC[0,:], 'ro-', label='u1')
ax5.plot(range(N_batch), u_MPC[1,:], 'bo-', label='u2')
ax5.set_xlabel('Time')
ax5.set_ylabel('u')
ax5.legend()

plt.tight_layout()


## Animate the MPC trajectory

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
import numpy as np

def plotRoverState(ax, x, y, psi, phiL, phiR, L, r_ratio):
    r = r_ratio * L / 2
    dx = (L / 2) * np.cos(psi + np.pi / 2)
    dy = (L / 2) * np.sin(psi + np.pi / 2)

    #Plot the rover
    ax.plot([x - dx, x + dx], [y - dy, y + dy], 'k-', linewidth=3)
    ax.plot(x, y, 'ko')
    ax.quiver(x, y, np.cos(psi), np.sin(psi), scale=20)
    
    wheelPosL = (x + dx, y + dy)
    wheelPosR = (x - dx, y - dy)

    circle1 = Circle(wheelPosL, r, edgecolor='b', facecolor='none')
    circle2 = Circle(wheelPosR, r, edgecolor='r', facecolor='none')
    ax.add_patch(circle1)
    ax.add_patch(circle2)

    ax.plot([wheelPosL[0], wheelPosL[0] + r * np.cos(phiL)], 
            [wheelPosL[1], wheelPosL[1] + r * np.sin(phiL)], 'b-')
    ax.plot([wheelPosR[0], wheelPosR[0] + r * np.cos(phiR)], 
            [wheelPosR[1], wheelPosR[1] + r * np.sin(phiR)], 'r-')

def calculate_wheel_positions(trajectory, L):
    N = len(trajectory)
    wheelL_pos = np.zeros((N, 2))
    wheelR_pos = np.zeros((N, 2))

    for i in range(N):
        x, y, psi, _, _ = trajectory[i]
        dx = (L / 2) * np.cos(psi + np.pi / 2)
        dy = (L / 2) * np.sin(psi + np.pi / 2)

        wheelL_pos[i] = [x + dx, y + dy]
        wheelR_pos[i] = [x - dx, y - dy]

    return wheelL_pos, wheelR_pos

def animate_trajectory_mpc(q_traj, u_traj, q_traj_OL, qf, L):
    # N = len(u_traj)
    n_k = np.shape(u_traj)[0]
    print(n_k)
    eps = 0.001
    for i in range(n_k):
        if ((q_traj[i,0] - qf[0])**2<= eps) & ((q_traj[i,1]-qf[1])**2<=eps) & ((q_traj[i,2]-qf[2])**2<=eps) & ((q_traj[i,3]-qf[3])**2<=eps) & ((q_traj[i,4]-qf[4])**2<=eps) & (u_traj[i,0]**2 <= eps) & (u_traj[i,1]**2<= eps):
            n_k = i
            break
    print(n_k)
    
    
    wheelL_pos, wheelR_pos = calculate_wheel_positions(q_traj, L)

    fig = plt.figure(figsize=(14, 6))
    ax1 = plt.subplot2grid((2, 4), (0, 0), colspan=2, rowspan=2)
    ax2 = plt.subplot2grid((2, 4), (0, 2))
    ax3 = plt.subplot2grid((2, 4), (0, 3))
    ax4 = plt.subplot2grid((2, 4), (1, 2))
    ax5 = plt.subplot2grid((2, 4), (1, 3))
    plt.tight_layout()

    def update(frame):
        ax1.clear()
        ax2.clear()
        ax3.clear()
        ax4.clear()
        ax5.clear()

        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax1.set_xlim([min(q_traj[:,0])-(r_cm+L_cm/2), max(q_traj[:,0])+(r_cm+L_cm/2)])
        ax1.set_ylim([min(q_traj[:,1])-(r_cm+L_cm/2), max(q_traj[:,1])+(r_cm+L_cm/2)])
        ax1.grid(True)
        ax1.set_aspect('equal')

        # Plot rover state
        #Show the full trajectory
        line_batch = ax1.plot(q_traj_batch[0,:], q_traj_batch[1,:], 'k-', alpha=0.5, label='Batch Trajectory')
        line_OL = ax1.plot(q_traj_OL[0, frame, :], q_traj_OL[1, frame, :], 'r.-', alpha=0.5, label='Open-loop')
        plotRoverState(ax1, *q_traj[frame], L, 0.3)
        ax1.legend([line_batch[0], line_OL[0]], ['Batch Trajectory', 'Open-loop'])

        # Plot trajectory
        x, y, psi, phiL, phiR = q_traj.T
        ax2.plot(range(n_k), x[:n_k], '.-', label='X-pos')
        ax2.plot(range(n_k), y[:n_k], '.-', label='Y-pos')
        ax2.plot(range(n_k), psi[:n_k], '.-', label='Heading')
        ax2.axvline(frame, 0, 1)
        ax2.legend()

        # Plot wheel angles
        ax3.plot(range(n_k), phiL[:n_k], '.-b', label="Lwhl θ")
        ax3.plot(range(n_k), phiR[:n_k], '.-r', label="Rwhl θ")
        ax3.axvline(frame, 0, 1)
        ax3.legend()

        # Plot wheel positions
        ax4.plot(range(n_k), wheelL_pos[:n_k, 0], '.-b', label="Lwhl x")
        ax4.plot(range(n_k), wheelL_pos[:n_k, 1], '.-b', label="Lwhl y")
        ax4.plot(range(n_k), wheelR_pos[:n_k, 0], '.-r', label="Rwhl x")
        ax4.plot(range(n_k), wheelR_pos[:n_k, 1], '.-r', label="Rwhl y")
        ax4.axvline(frame, 0, 1)
        ax4.legend()

        # Plot inputs
        ax5.plot(range(n_k), u_traj[:n_k, 0], '.-b', label="uL")
        ax5.plot(range(n_k), u_traj[:n_k, 1], '.-r', label="uR")
        ax5.axvline(frame, 0, 1)
        ax5.legend()

        return ax1, ax2, ax3, ax4

    ani = FuncAnimation(fig, update, frames=n_k, repeat=False)
    return ani


ani = animate_trajectory_mpc(q_MPC.T, u_MPC.T, q_MPC_openloop, qf, L_cm)
# Display animation in Jupyter
from IPython.display import HTML
HTML(ani.to_jshtml())
 